# Overview

In this exercise, we will be performing analysis on CT examinations of the abdomen performed in patients with renal carcinoma (e.g. kidney cancer). Our goal with be to perform a few simple preprocessing steps in preparation for training a machine learning algorithm. The data for this exercise is a small sample from the KiTS 2019 MICCAI Challenge (https://kits19.grand-challenge.org/home/).

## Data

We will begin by downloading the data, and taking a look at some of its organization here:

In [0]:
# --- Download the data
!wget -O ex1.zip "https://www.dropbox.com/s/v2f14er8yku0wwf/ex1.zip?dl=1"
!mkdir /data
!unzip -o ex1.zip -d /data

In [0]:
# --- List directory contents
!ls -al /data/ex1

# --- List contents of a single folder
!ls -al /data/ex1/case_00010/

The raw data we will be using in the exercise are 3D CT scan volumes of the abdomen. The data is stored as a 4D array of size: "Z" by "Y" by "X" by 1 (e.g. the fourth "dimension" is always one for this particular dataset). This format, while a bit unusual, is the default order used by most machine learning libraries such as Tensorflow. Let's take a look here:

In [0]:
# --- Use NumPy to load data files
import numpy as np

# --- Note that the data consists of 4D volumes
dat = np.load('/data/ex1/case_00010/dat.npy')
print(type(dat))
print(dat.shape)

# --- Load corresponding voxel sizes
dim = np.load('/data/ex1/case_00010/dim.npy')
print(type(dim))
print(dim.shape)
print(dim)

## Visualization

At various points of this exercise, it will be crucial to visualize the data. To do so, feel free to use this simple method to show any given single slice of the 3D volume (of course you may add and/or embellish this method as you like).

In [0]:
# --- Visualize using pylab
import pylab

def show_2d(im, title=None, figsize=(7, 7)):
    """
    Method to show 2D image 
    
    """
    pylab.figure(figsize=figsize)
    pylab.axis('off')
    pylab.imshow(np.squeeze(im), cmap=pylab.cm.gist_gray, vmin=-240, vmax=240)
    
    if title is not None:
        pylab.title(title)

# --- Show the 25th "slice" of the dat volume
show_2d(dat[25])

## Voxel Size and Orientation

Each 3D "pixel" in our volume matrix is known as a **voxel**. Voxels have a size (in mm) in each Z, Y and X dimension. In our particular dataset, based on acquisition parameters, the image volumes are isotropic in the XY-plane (e.g. the X and Y voxel sizes are the same) but different in the Z-axis. 

Voxel size is an important consideration when re-orienting (e.g. flipping) the raw volume matrices into different planes. More specifically in the world of medical imaging, the same data volume can be viewed in three dominant orientations: axial, coronal and sagittal.

See diagram here:

![Orientation](https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/Human_anatomy_planes%2C_labeled.jpg/220px-Human_anatomy_planes%2C_labeled.jpg)

Note the following conventions:

* Z-axis = up-down axis
* Y-axis = forward-backward axis
* X-axis = left-right axis
* XY-plane = axial
* ZX-plane = coronal
* ZY-plane = sagittal

Because of differences in voxel size in the three planes, if we naively rotate our image matrix in the three planes, the images will look *normal* in the XY plane only but *distorted* in the ZY or ZX planes.

See examples here:

In [0]:
# --- Axial image (normal)
axi = dat[25]
show_2d(axi, 'orient = axial')

In [0]:
# --- Coronal image (distorted/compressed)
cor = dat[:, 256]
show_2d(cor, 'orient = coronal')
print(cor.shape)

In [0]:
# --- Sagittal image (distorted/compressed)
sag = dat[:, :, 256]
show_2d(sag, 'orient = sagittal')
print(sag.shape)

To correct this distortion, we would need to resample (zoom) the volume as shown here while **accounting for voxel size**:

In [0]:
from scipy import ndimage

# --- Calculate ratio of isotropic image
zoom = [dim[0] / dim[1], 1, 1]

# --- Resample to isotropic
print('Before zoom: ', cor.shape)
cor_ = ndimage.zoom(cor, zoom=zoom, order=1)
print('After zoom:', cor_.shape)

# --- Show new image
show_2d(cor_)

Much better! Now that we have accounted for the differences in voxel size, we have properly "uncompressed" our coronal image.

# Exercise 1

CT scans of the abdomen are highly varied with regards to field of view and patient positioning. Thus, the first part of this algorithm is to generate bounding cube crops of the right and left hemi-abdomen (e.g. each containing the right and left kidney, respectively). By standardizing the field of view, these cropped volumes will make it much easier for our downstream deep learning networks to perform subsequent localization and classification tasks.

## Resampling

To begin, let us resample all of our 3D volumes into a uniform shape. You will notice that all of our raw CT volumes are of different matrix shapes. To train our algorithm we need to resample our data to a uniform 256 x 256 x 256 (Z x Y x X) shape. Importantly the cube volume must also be **isotropic**, in other words the same **voxel size** in all the dimensions, with no distortion in any plane.

Based on this, the first task to perform is the following: given an arbitrary 3D image volume of shape Z x Y x X, and corresponding voxel dimensions in each direction (in mm), write an algorithm to generate a fixed 256 x 256 x 256 cube with the **same voxel size** in each direction. Try to keep the algorithm as generic as possible---in our three examples, the Z- voxel size is always larger than the X-/Y- voxel size (and the X-/Y- voxel sizes are the same) but do not assume that this is always the case. As needed, you may pad the original volume with zeros in up to **two dimensions** (if you need to pad in all three directions then reduce the padding in all directions until at least one dimension does not need to be padded anymore).

Write your final algorithm by filling in the method below; feel free to use additional code blocks as needed to test and brainstorm.

In [0]:
def resample_to_256_cube(vol, dims):
    """
    Method to resample provided volume to 3D isotropic 256 x 256 x 256 matrix
    
    :params
    
      (np.ndarray) vol  : a 4D matrix of size Z * Y * X * 1
      (np.ndarray) dims : a 3-element vector of voxel size (in mm)
      
    """

# Exercise 2

Now that we have 256 x 256 x 256 cubes, we can train a CNN algorithm to generate the desired 3D crops extending along the boundaries of the right and left hemi-abdomen. The following images show examples of the final desired right- and left- crops in the axial, coronal and sagittal planes: 

## Crops (axial plane)

![axial](https://raw.githubusercontent.com/CAIDMRes/kits_test/master/pngs/kits_axi.png)

## Crops (coronal plane)

![coronal](https://raw.githubusercontent.com/CAIDMRes/kits_test/master/pngs/kits_cor.png)

## Crops (sagittal plane)

![sagittal](https://raw.githubusercontent.com/CAIDMRes/kits_test/master/pngs/kits_sag.png)



To generate these 3D bounding cube crops, we will train a 2D slice-by-slice CNN binary classifier to determine whether any given "slice" of data contains **any portion** of the region of interest shown above. For example in the axial plane, the algorithm would return a prediction of 1 (True) if the slice is at a level that includes any part of the hemi-abdomen (green region) and a prediction of 0 (False) if the slice is at a level above (e.g. lungs) or below ( e.g. pelvis) the region of interest. The same type of information will also be returned by feeding in slices from any of the three orthogonal planes. Because our input volume is a 256 x 256 x 256 cube, a series of predictions along any given single plane can be represented as a 256-element vector.

Let us download and look at some example data here:

In [0]:
# --- Download additional data
!wget -O ex2.zip "https://www.dropbox.com/s/sseq8g00ebni6iz/ex2.zip?dl=1"
!unzip -o ex2.zip -d /data

In [0]:
# --- List directory contents
!ls -al /data/ex2

# --- List contents of a single folder
!ls -al /data/ex2/case_00214/

In [0]:
# --- Load remaining data
dat = np.load('/data/ex2/case_00214/dat.npy')
print(type(dat))
print(dat.shape)

# --- Load corresponding prediction vectors
pred_axi = np.load('/data/ex2/case_00214/pred_axi.npy')
pred_cor = np.load('/data/ex2/case_00214/pred_cor.npy')
pred_sag = np.load('/data/ex2/case_00214/pred_sag.npy')

print(type(pred_axi))
print(pred_axi.shape)

Using this data, the second task to perform is the following: given three perpendicular prediction vectors (each of size 256), generate a final 3D mask (256 x 256 x 256) corresponding to the two areas of interest shown above. The mask you create should be binary (e.g. 1 = region of interest, 0 = background), and should delineate the two ROIs for the right and left hemi-abdomen.

**Important**: Note that the prediction vectors are generated by a CNN, and thus will have a degree of noise. In other words, occassionally there will be false positive and false negative predictions. Therefore as needed you may postprocess the prediction vectors so that the two most dominant areas of interest are properly identified.

As an example, see sagittal prediction below---while there are two "dominant" areas, occasionally there is a mistake (~5%) in the algorithm prediction. 


In [0]:
print(pred_sag)

Write your final algorithm by filling in the method below. Feel free to test your algorithm by overlaying the mask on your original volumes and visualizing. Your final results should be similar to the ROIs shown above.

In [0]:
def create_cube_mask(pred_axi, pred_cor, pred_sag):
    """
    Method to generate 3D mask of the R-/L- hemiabdomen using orthogonal 
    prediction vectures generated by a CNN.
    
    :params
    
      (np.ndarray) pred_axi : 256-element prediction vector in the axial plane
      (np.ndarray) pred_cor : 256-element prediction vector in the coronal plane
      (np.ndarray) pred_sag : 256-element prediction vector in the sagittal plane
      
    :return
    
      (np.ndarray) mask : 3D binary mask of size 256 x 256 x 256 contained the 
                          filled two dominant ROIs

    """